# Chapter10. 시퀀스 해킹, 해시, 슬라이스

## 10.1 Vector: 사용자 정의 시퀀스 형
우리의 전략은 상속이 아니라 구성을 이용해서 벡터를 구현하는 것이다. 요소들을 실수형 배열에 저장하고, 벡터가 불변 균일 시퀀스처럼 작동하게 미]ㅏㄴ들기 위해 메서드를 구현한다.  
그러나 시퀀스 메서드를 구현하기 전에, 앞에서 구현한 Vector2d 클래스와 호화성이 높은 기본 Vectr 클래스를 먼저 만들어보자

## 10.2 Vector 버전#1: Vector2d 호환

In [1]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'

    def __init__(self, components):
        print()
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]           # array인거 들키고 싶지 않아요
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool((abs(self)))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [8]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [9]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [7]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

## 10.3 프로토콜과 덕 타이핑

시퀀스 객체를 생성하기 위해서는 `__len__()`과 `__getitem__()` 메서드를 정의하면 된다. 그것을 파이썬의 시퀀스 프로토콜이라고 한다.

In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

위의 예제의 FrenchDeck클래스는 시퀀스 프로토콜을 구현하므로 파이썬에서 제공하는 여러 기능을 활용할 수 있다. 코드 어디에도 시퀀스 프로토콜을 따른다고 정의한 곳은 없다. 해당 클래스가 object를 상속하고있지만 모든 파이썬 프로그래머들은 이 코드를 보면 시퀀스라는 것을 알 수 있다. **이 클래스가 시퀀스처럼 동작하기 때문에 시퀀스인 것이다.**

프로토콜은 비공식적이고 강제로 적용되는 사항이 아니므로 클래스가 사용되는 특정 환경에 따라 프로토콜의 일부만 구현할 수도 있다. 예를 들어 반복을 지원하려면 `__getitem__()` 메서드만 구현하면 되며, `__len__()` 메서드를 구현할 필요는 없다.

## 10.4 Vector 버전 #2: 슬라이스 가능한 시퀀스

In [8]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1] 
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        return self._components[index]

In [4]:
v1 = Vector([3, 4, 5])
len(v1)

array('d', [3.0, 4.0, 5.0])


3

In [9]:
v1

Vector([3.0, 4.0, 5.0])

In [5]:
v1[0], v1[-1]

(3.0, 5.0)

In [6]:
v7 = Vector(range(7))
v7[1:4]

array('d', [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0])


array('d', [1.0, 2.0, 3.0])

### 10.4.1 슬라이싱의 작동방식

In [14]:
class MySeq:
    def __getitem__(self, index):
        return index

s = MySeq()
s[1]

1

In [15]:
s[1:4]

slice(1, 4, None)

In [16]:
s[1:4:2]

slice(1, 4, 2)

In [17]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [18]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

In [24]:
repr(slice)

"<class 'slice'>"

In [23]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [26]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [31]:
'ABCDE'[:10:2], 'ABCDE'[0:5:2]

('ACE', 'ACE')

In [32]:
slice(-3, None, None).indices(5)

(2, 5, 1)

In [33]:
'ABCED'[-3:], 'ABCDE'[2:5:1]

('CED', 'CDE')

In [45]:
from array import array
import reprlib
import math
import numbers

class VectorV2:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

In [46]:
v7 = VectorV2(range(7))
v7[-1]

6.0

In [40]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [41]:
v7[-1]

6.0

In [42]:
v7[1,2]     # 다차원 인덱싱 지원하지 않음

TypeError: Vector indices must be integers

In [47]:
from array import array
import reprlib
import math
import numbers

class VectorV3:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

In [48]:
v = VectorV3(range(10))
v.x

0.0

In [58]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [56]:
v.x = 10
v.x

10

In [52]:
v.x, v.y, v.z

(10, 1.0, 2.0)

In [54]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [73]:
class PrivateC():
    def __init__(self, x,y):
        self._x = x
        self.y = y


c = PrivateC(1, 2)
c._x


1

In [74]:
c._x = 10

In [76]:
c._x, c.y

(10, 2)

# NOTE

In [83]:
items = [[10, 20], [30, 40], [50, 60]]
n = items[0][1]
[n:= n+y for x, y in items]

[40, 80, 140]

In [96]:
a = ['a', 'b']
b = ['c', 'd']
z = zip(a, b)

for i in range(2):
    for a, b in zip(a, b):
        print(i, a, b)

print('-----')
for i in range(2):
    for a, b in z:
        print(i, a, b)     # 왜 다른거임?!

0 a c
0 b d
1 b d
-----
0 a c
0 b d


In [101]:
def test_generator():
    yield 1
    yield 2
    yield 3

gen = test_generator()
type(gen)

generator

In [99]:
a = []
for i in gen:
    a.append(i)
    print(i)